David James Fulton, Durham Univeristy, 4th year project 2016

# Two Parameter Quantum Annealing with Multi-Body Interactions

Summary:
Quantum annealing works by preparing a simple quantum system in its lowest energy state and slowly
transforming it into a different quantum system where the lowest energy state maps to the solution of
a hard problem. This protocol has many potential real world applications and is the ones used in the
devices developed by D-Wave Systems Inc. The purpose of this project is to study ways to perform
this protocol differently by changing two independent parameters using the coupling gadgets
proposed in (1), rather then one. Most of the work on this project will consist of using numerical
techniques to simulate the quantum annealing protocol.

References:

[1] Chancellor, N., Zohren, S., Warburton, P. A., Circuit design for multi-body interactions in
superconducting quantum annealing system with applications to a scalable architecture,
arXiv:1603.0952 (2016).

[2] Crosson, E. et. al. Different Strategies for Optimization Using the Quantum Adiabatic Algorithm
arXiv:1401.7320 (2014).

[3] http://www.dwavesys.com/resources/publications D-Wave publications webpage accessed: April
27, 2016

The base hamiltonian
\begin{equation}
H_{B} = \sigma_{x}
\end{equation}

The problem Hamiltonian
\begin{equation}
H_{P} =\sigma_{z}
\end{equation}

using
\begin{equation}
A(s) = 1-s
\end{equation}
and
\begin{equation}
B(s) = s
\end{equation}

\begin{equation}
H(s) = -A(s)H_{B}+B(s)H_{P}
\end{equation}
so

\begin{equation}
H(s) = -(1-s)H_{B} + s H_{P}
\end{equation}

\begin{equation}
\Psi_{i+1} = (1-iH(s_{i}))\Psi_{i}dt
\end{equation}

In [ ]:
import scipy.linalg as LA
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt


In [ ]:
Hb = np.matrix('0. 1.; 1. 0.')
Hp = np.matrix('1. 0.; 0. -1.')
x0 = 1./2**0.5 * np.matrix('1. ;-1.')+1j* np.matrix('0. ;0.')
x1 = 1./2**0.5 * np.matrix('1. ; 1.')+1j* np.matrix('0. ;0.')
# it starts in state x0, the ground state of the initial hamiltonian 

def H(s):
    h = -(1-s)*Hb + s*Hp
    return h

ds = 0.01
ss = np.arange(0,1,ds)
T = 10
dt = ds*T
state = [x0]
for i in range(len(ss)):
    state.append(state[-1]-1.j*H(ss[i])*state[-1]*dt)
print state[-1]
print abs(state[-1][0])**2+abs(state[-1][1])**2

# Week 2

This method I have been using isn't very good and I got ahead of myself here. The integration is only to first order in t and I think this may be why it is blowing up. 

I have a new goal now based on the meeting with Viv and Nick (Vick). I want to see how the energy eigenvalues change with the annealing schedule and I will do the evolution using their methods.



### Tutorial from week 2

In [ ]:
%matplotlib inline
img1=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Tutorial Photos\week2_1.jpg")
img2=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Tutorial Photos\week2_2.jpg")
img3=mpimg.imread(r"C:\Users\User\Documents\FIZZIX\4th Year\Project\Tutorial Photos\week2_3.jpg")
plt.figure(figsize=(10,6)); plt.imshow(img1)
plt.figure(figsize=(10,6)); plt.imshow(img2)
plt.figure(figsize=(10,6)); plt.imshow(img3)
plt.show('all')




### Tracing the energy eigenvalues over the anneal schedule 

In [ ]:
'''This cell is to plot how the energy eigenvalues change throughout the annealing process for the
hamiltonian and annealing schedule above'''
#initiate store for the data 
eigvals = []

# do the eigenvalue calculations
for i,s in enumerate(ss):
    eigvals.append(LA.eigvalsh(H(s)))
eigvals = np.asarray(eigvals)

# plot
plt.plot(ss,eigvals[:,1])
plt.plot(ss,eigvals[:,0])
plt.xlabel('s')
plt.ylabel('$\lambda$')
plt.show()

This looks like what I would have expected. Perhaps I should move on to do the same thing with 2 coupled qubits


In [ ]:
'''In this cell we aim to do the same thing as above, trace the eigenvalues of the hamiltonian
throught the annealing process, but now with 2 coupled qubits'''

# define some standard contants that we will need
I2 =  np.eye(2)
sigma_x = np.matrix('0. 1.; 1. 0.')
sigma_z = np.matrix('1. 0.; 0. -1.')

# define the hamiltonians 
h_i = [-8,0.5]
J = 2.

Hb_2 = np.kron(sigma_x,I2)+np.kron(I2,sigma_x)
Hp_2 = h_i[0]*np.kron(sigma_z,I2)+h_i[1]*np.kron(I2,sigma_z) + J*np.kron(sigma_z,I2)*np.kron(I2,sigma_z)

def H_2(s):
    h = -(1-s)*Hb_2 + s*Hp_2
    return h

# create the data store and trace the eigenvalues
eigvals = []
for i,s in enumerate(ss):
    eigvals.append(LA.eigvalsh(H_2(s)))
eigvals = np.asarray(eigvals)

# output and plot
print 'last set of eigenvalues were:',eigvals[-1]

for i in range(eigvals.shape[1]):
    plt.plot(ss,eigvals[:,i])
plt.xlabel('s')
plt.ylabel('$\lambda$')
plt.show()


Now that we have done this I think it may be time to move on to try and run the annealing schedule. We may want to revisit this in a while but for now I think we can move on as the only steps to make this more complicated are to (change J and $h_{i}$ terms/add a third qubit)


Note: we used the np.kron function in the previus section. This does the tensor product of two matrices

### Some functions to use later

In [ ]:
# define and test a function
def Diagonaliser(M):
    '''This function takes a hermitian matrix and return the unitary transform matrices to change to and from the eigenbasis
    as well as the diagonalised form of this matrix'''
    eigvals, eigvecs = np.linalg.eigh(M)
    zers = np.zeros((len(eigvals),len(eigvals)))
    P_in = np.matrix(zers)
    P = np.matrix(zers)
    for i in range(len(eigvals)):
        P[:,i] = eigvecs[:,i]
        P_in[i,:] = np.transpose(eigvecs[:,i])

    return eigvals,P,P_in

test = np.matrix(np.random.random((4,4)))
Diagonaliser(test)

Remembering now that $M = PDP^{-1}$ we can start applying the transformation.

But first below we will run a little test

In [ ]:
# define and test a function
def make_diag(d):
    '''this is a function to make a diagonal matrix given a series of values to go on the diagonal'''
    zers = np.zeros((len(d),len(d)))+ 1.j*np.zeros((len(d),len(d)))
    np.fill_diagonal(zers,d)
    return np.matrix(zers)


M = 2*sigma_x-5*sigma_z
v = np.matrix('2.3; 1.1')
d,P,P_in = Diagonaliser(M)

print M*v
print P*make_diag(d)*P_in*v

### Running the annealing process first for 1 qubit then 2

in order to map transform a qubit state from one instance to the next we can use the operation:
\begin{equation}
\Psi_{i+1} = Pe^{-iH(t_{i})\delta t}P^{-1}\Psi_{i}
\end{equation}

Below we will run this on the case of a single qubit with applied problem and base hamiltonian chan ging throughout the annealing schedule

In [ ]:
# define the hamiltonians for the base and the problem
Hb = np.matrix('0. 1.; 1. 0.')
Hp = 3*np.matrix('1. 0.; 0. -1.')

def H(s):
    '''returns the hamiltonian at any point during the annealing schedule'''
    h = -(1-s)*Hb + s*Hp
    return h

#calulate the ground state
x0 = np.transpose(np.linalg.eigh(H(0))[1][0])
x0 = x0+0.j*x0

# define the paramters that define the annealing
ds = 0.0001
ss = np.arange(0,1,ds)
T = 100
dt = ds*T

# assign data stores for the anneal. We will keep energy eigenvalue and state
state = [x0]
eigenvals =[]

# run the anneal storing both state and energy eigenvalues
for i in ss:
    d,P,P_in = Diagonaliser(H(i))
    eigenvals.append(d)
    state.append(P*make_diag(np.exp(-1.j*d*dt))*P_in*state[-1])
eigenvals = np.asarray(eigenvals)

#plot and print the important results for comparison
print 'last set of eigenvalues were:',eigenvals[-1]
plt.figure()
plt.plot(ss,eigenvals[:,0])
plt.plot(ss,eigenvals[:,1])
plt.show()
print 'initial x0:','\n', x0,'\n\n', 'new x0:','\n', np.transpose(np.linalg.eigh(H(1))[1][0])
print '\n\n','final state:', '\n',abs(state[-1])
print '\nfinal state amplitude:', state[-1].getH()*state[-1]
print '\nprobability of being measured in lowest energy state: ', ((P_in*state[-1])[d.argmin()]*(P_in*state[-1])[d.argmin()].conj()).real


Seems to be going well. Now lets try and move on to do the same annealing with 2 qubits instead of just one. All of the maths and code will be largely the same

\begin{equation}
H_{B} = (\sigma_{1}^{x}\otimes\mathcal{1}_{2})
+(\mathcal{1}_{1}\otimes\sigma_{2}^{x})
\end{equation}



\begin{equation}
H_{P} = h_{1}(\sigma_{1}^{z}\otimes\mathcal{1}_{2})
+h_{2}(\mathcal{1}_{1}\otimes\sigma_{2}^{z})
+J_{12}(\sigma_{1}^{z}\otimes\mathcal{1}_{2})(\mathcal{1}_{1}\otimes\sigma_{2}^{z})
\end{equation}

\begin{equation}
H(s) = -A(s)H_{B} + B(S)H_{P}
\end{equation}

In [ ]:
# define the base and problem hamiltonians
h_i = [-8,0.5]
J = 2.

Hb_2 = np.kron(sigma_x,I2)+np.kron(I2,sigma_x)
Hp_2 = h_i[0]*np.kron(sigma_z,I2)+h_i[1]*np.kron(I2,sigma_z) + J*np.kron(sigma_z,I2)*np.kron(I2,sigma_z)

def H_2(s):
    '''This function returns the hamiltonain at any point in the annealing schedule'''
    h = -(1-s)*Hb_2 + s*Hp_2
    return h

# find the ground state of the base hamiltonian 
eigvals0,eigvecs0 = np.linalg.eigh(H_2(0))
x0 = eigvecs0[:,eigvals0.argmin()]
x0 = x0+0.j*x0

# set up variables for the anneal
ds = 0.0001
ss = np.arange(0,1,ds)
T = 10000 # this is a scale factor to allow the rate of the anneal to be changed
dt = ds*T

# set up data stores for the anneal whilst setting the inital state to be th ground state on the base hamiltonian
state = [x0]
eigenvals =[]

# carry out the anneal whilst storing the state and the energy eigenvalues thoughout the process
for i in ss:
    d,P,P_in = Diagonaliser(H_2(i))
    eigenvals.append(d)
    state.append(P*make_diag(np.exp(-1.j*d*dt))*P_in*state[-1])

eigenvals = np.asarray(eigenvals)

In [ ]:
%matplotlib inline
print 'last set of eigenvalues were:',eigenvals[-1]
plt.figure()
for i in range(len(eigenvals[0,:])):
    plt.plot(ss,eigenvals[:,i])
plt.show()

eigvalsn0,eigvecsn0 = np.linalg.eigh(H_2(1.))
d,P,P_in = Diagonaliser(H_2(1.))

xn0 = eigvecsn0[:,eigvalsn0.argmin()]
print 'initial x0:','\n', x0,'\n\n', 'new x0:','\n', xn0
print '\n\n','final state:', '\n',abs(state[-1]),
print '\n\n','Final state amplitude: ',state[-1].getH()*state[-1]
chan = ((P_in*state[-1])[d.argmin()]*(P_in*state[-1])[d.argmin()].conj()).real
print '\nProbability of being measured in lowest energy state: %s' %chan



### Extra considerations about the accuracy of these results

In [ ]:
H_2(1)

I've noticed that the linalg function I have been using to calculate the eigenvalues and eigenvectors aren't prefect. They have numerical errors in them for most values I have but in. This is demonstrated in the code I have written below. If you set s=1 then it works out perfect with $H(s=1)-E_{k})\vec{v_{k}}=\vec{0}$ but for every other value I have tested it on it ends up with vector elements of approximately $10^{-16}$. It also seems to struggle with degenerate eigenvalues. Demonstrated with the s = 0 case. This should have a repeat root of $E_{k} = 0$ but this is not the case.

In [ ]:
import scipy.sparse.linalg
h2 = H_2(0.)
eigval,eigvec = np.linalg.eig(h2)
print 'eigenvalues: ', eigval, '\n-----------------------------\n'
for i in range(4):
    print 'eigenvalue E: ',eigval[i],'\n'
    print 'eigenvector v: ', '\n', eigvec[:,i],'\n'
    print 'product of (H-E)v:','\n', (h2-eigval[i]*np.eye(4))*eigvec[:,i], '\n-----------------------------\n'